In [1]:
from transformers import TFDistilBertForQuestionAnswering, DistilBertTokenizer 
import tensorflow as tf

MODEL_NAME = 'distilbert-base-cased-distilled-squad'

tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained('./{}_tokenizer/'.format(MODEL_NAME))

try:
    model = TFDistilBertForQuestionAnswering.from_pretrained(MODEL_NAME)
except:
    model = TFDistilBertForQuestionAnswering.from_pretrained(MODEL_NAME, from_pt=True)
    
# Define TF Signature
@tf.function(
  input_signature=[
      {
          "input_ids": tf.TensorSpec((None, None), tf.int32, name="input_ids"),
          "attention_mask": tf.TensorSpec((None, None), tf.int32, name="attention_mask")       
      }
  ]
)
def serving_fn(input):
    return model(input)

model.save_pretrained("./{}".format(MODEL_NAME), saved_model=True, signatures={"serving_default": serving_fn})

2023-05-16 03:29:51.226311: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-16 03:29:51.338124: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-16 03:29:51.341948: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-16 03:29:51.341960: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


INFO:tensorflow:Assets written to: ./distilbert-base-cased-distilled-squad/saved_model/1/assets


INFO:tensorflow:Assets written to: ./distilbert-base-cased-distilled-squad/saved_model/1/assets


In [2]:
!ls -l {MODEL_NAME}

total 254796
-rw-r--r-- 1 root root       569 May 16 03:30 config.json
drwxr-xr-x 3 root root      4096 May 16 03:30 saved_model
-rw-r--r-- 1 root root 260895720 May 16 03:30 tf_model.h5


In [3]:
!ls -l {MODEL_NAME}/saved_model/1

total 4972
drwxr-xr-x 2 root root    4096 May 16 03:30 assets
-rw-r--r-- 1 root root      57 May 16 03:30 fingerprint.pb
-rw-r--r-- 1 root root   79098 May 16 03:30 keras_metadata.pb
-rw-r--r-- 1 root root 4996317 May 16 03:30 saved_model.pb
drwxr-xr-x 2 root root    4096 May 16 03:30 variables


In [4]:
!ls -l {MODEL_NAME}_tokenizer

total 220
-rw-r--r-- 1 root root    125 May 16 03:29 special_tokens_map.json
-rw-r--r-- 1 root root    427 May 16 03:29 tokenizer_config.json
-rw-r--r-- 1 root root 213450 May 16 03:29 vocab.txt


In [5]:
asset_path = '{}/saved_model/1/assets'.format(MODEL_NAME)

!cp {MODEL_NAME}_tokenizer/vocab.txt {asset_path}

In [6]:
!ls -l {MODEL_NAME}/saved_model/1/assets

total 212
-rw-r--r-- 1 root root 213450 May 16 03:30 vocab.txt


In [7]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

:: loading settings :: url = jar:file:/root/miniconda3/envs/sp/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-aa0a5bb8-7683-41ed-853b-1c929e11a6cf;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.4.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.16.0 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.guava#failureaccess;1.0.1 

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
    .master("local[*]")\
    .config("spark.driver.memory", "16G")\
    .config("spark.driver.maxResultSize", "0")\
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")\
    .config("spark.kryoserializer.buffer.max", "2000m")\
    .config("spark.jsl.settings.pretrained.cache_folder", "sample_data/pretrained")\
    .config("spark.jsl.settings.storage.cluster_tmp_dir", "sample_dataorage")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.4.1")\
    .getOrCreate()

In [8]:
from sparknlp.annotator import *
from sparknlp.base import *

MODEL_NAME = 'distilbert-base-cased-distilled-squad'

spanClassifier = DistilBertForQuestionAnswering.loadSavedModel(
     '{}/saved_model/1/'.format(MODEL_NAME),
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(512)

In [9]:
spanClassifier.write().overwrite().save("./{}_spark_nlp".format(MODEL_NAME))

In [10]:
!rm -rf {MODEL_NAME}_tokenizer {MODEL_NAME}

In [11]:
!ls -l {MODEL_NAME}_spark_nlp

total 259516
-rw-r--r-- 1 root root 265735555 May 16 03:30 distilbert_classification_tensorflow
drwxr-xr-x 4 root root      4096 May 16 03:30 fields
drwxr-xr-x 2 root root      4096 May 16 03:30 metadata


In [12]:
document_assembler = MultiDocumentAssembler() \
    .setInputCols(["question", "context"]) \
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = DistilBertForQuestionAnswering.load("./{}_spark_nlp".format(MODEL_NAME))\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

example = spark.createDataFrame([["What's my name?", "My name is Clara and I live in Berkeley."]]).toDF("question", "context")
result = pipeline.fit(example).transform(example)

result.select("answer.result").show(1, False)

+-------+
|result |
+-------+
|[Clara]|
+-------+

